In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
events_agg = pd.read_csv('events_geo_taxi.csv') #load events data

In [4]:
events_agg.head()

,Unnamed: 0,Unnamed: 0.1,id,datetime_local,title,type,venue.id,venue.location.lat,venue.location.lon,venue.name,venue.postal_code,geometry,LocationID,borough
0,1,1,4442708,2018-11-08T11:00:00,Big Apple Circus - New York,family,3900,40.7726,-73.9841,Damrosch Park,10023,POINT (-73.9841 40.7726),142,Manhattan
1,12,12,4547368,2018-11-08T19:00:00,Plot Points in Our Sexual Development - New York,broadway_tickets_national,8948,40.7736,-73.9837,Lincoln Center Theater - Claire Tow Theater,10023,POINT (-73.9837 40.7736),142,Manhattan
2,14,14,3991302,2018-11-08T19:00:00,My Fair Lady - New York,broadway_tickets_national,2953,40.7731,-73.9822,Lincoln Center Theater-Vivian Beaumont,10023,POINT (-73.98220000000001 40.7731),142,Manhattan
3,15,15,4442706,2018-11-08T19:00:00,Big Apple Circus - New York,family,3900,40.7726,-73.9841,Damrosch Park,10023,POINT (-73.9841 40.7726),142,Manhattan
4,30,30,4596170,2018-11-08T19:00:00,OpenICE,concert,457975,40.7734,-73.9846,"The Vincent Astor Gallery, The New York Public...",10023,POINT (-73.9846 40.7734),142,Manhattan


In [5]:
events_agg['LocationID'].nunique()

41

In [3]:
events_agg['datetime_local'] = pd.to_datetime(events_agg['datetime_local'])

In [4]:
events_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14583 entries, 0 to 14582
Data columns (total 14 columns):
Unnamed: 0            14583 non-null int64
Unnamed: 0.1          14583 non-null int64
id                    14583 non-null int64
datetime_local        14583 non-null datetime64[ns]
title                 14583 non-null object
type                  14583 non-null object
venue.id              14583 non-null int64
venue.location.lat    14583 non-null float64
venue.location.lon    14583 non-null float64
venue.name            14583 non-null object
venue.postal_code     14583 non-null int64
geometry              14583 non-null object
LocationID            14583 non-null int64
borough               14583 non-null object
dtypes: datetime64[ns](1), float64(2), int64(6), object(5)
memory usage: 1.6+ MB


In [5]:
events_agg['hour'] = events_agg['datetime_local'].dt.hour

In [6]:
events_agg['nightevent'] = np.where(events_agg['hour'] >= 19, 1, 0)

In [7]:
events_agg.head()

,Unnamed: 0,Unnamed: 0.1,id,datetime_local,title,type,venue.id,venue.location.lat,venue.location.lon,venue.name,venue.postal_code,geometry,LocationID,borough,hour,nightevent
0,1,1,4442708,2018-11-08 11:00:00,Big Apple Circus - New York,family,3900,40.7726,-73.9841,Damrosch Park,10023,POINT (-73.9841 40.7726),142,Manhattan,11,0
1,12,12,4547368,2018-11-08 19:00:00,Plot Points in Our Sexual Development - New York,broadway_tickets_national,8948,40.7736,-73.9837,Lincoln Center Theater - Claire Tow Theater,10023,POINT (-73.9837 40.7736),142,Manhattan,19,1
2,14,14,3991302,2018-11-08 19:00:00,My Fair Lady - New York,broadway_tickets_national,2953,40.7731,-73.9822,Lincoln Center Theater-Vivian Beaumont,10023,POINT (-73.98220000000001 40.7731),142,Manhattan,19,1
3,15,15,4442706,2018-11-08 19:00:00,Big Apple Circus - New York,family,3900,40.7726,-73.9841,Damrosch Park,10023,POINT (-73.9841 40.7726),142,Manhattan,19,1
4,30,30,4596170,2018-11-08 19:00:00,OpenICE,concert,457975,40.7734,-73.9846,"The Vincent Astor Gallery, The New York Public...",10023,POINT (-73.9846 40.7734),142,Manhattan,19,1


In [8]:
events_agg = events_agg[['id','LocationID','hour','nightevent']]

In [9]:
tz_events_agg = events_agg.groupby(['LocationID'])['nightevent'].agg(['sum','count'])

In [10]:
tz_events_agg.reset_index(inplace=True)
tz_events_agg.head()

,LocationID,sum,count
0,4,54,54
1,13,22,54
2,41,15,21
3,42,0,1
4,43,300,418


In [11]:
taxi_shapes = gpd.read_file('taxi_zones.shp')

In [12]:
taxi_shapes.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527103 192536.0856972019,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"(POLYGON ((1033269.243591294 172126.0078125, 1..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506663 256767.6975403726,..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968601 203714.0759887695,..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932606 144283.335850656, ..."


In [13]:
events_shp = pd.merge(taxi_shapes,tz_events_agg,how='left',left_on='LocationID',right_on='LocationID')

In [14]:
events_shp.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,sum,count
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527103 192536.0856972019,...",NaN,NaN
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"(POLYGON ((1033269.243591294 172126.0078125, 1...",NaN,NaN
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506663 256767.6975403726,...",NaN,NaN
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968601 203714.0759887695,...",54.0,54.0
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932606 144283.335850656, ...",NaN,NaN


In [15]:
events_shp.rename(columns={"sum": "night_events", "count": "all_events"},inplace=True)

In [17]:
events_shp['night_events'].fillna(0,inplace=True)
events_shp['all_events'].fillna(0,inplace=True)

In [18]:
events_shp.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,night_events,all_events
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527103 192536.0856972019,...",0.0,0.0
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"(POLYGON ((1033269.243591294 172126.0078125, 1...",0.0,0.0
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506663 256767.6975403726,...",0.0,0.0
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968601 203714.0759887695,...",54.0,54.0
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932606 144283.335850656, ...",0.0,0.0


In [20]:
events_shp.to_file(driver='ESRI Shapefile',filename='events_shp')